In [67]:
import glob
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [68]:
def load_image(index):
    global img, original_image
    cv.destroyAllWindows()
    img = cv.imread(images[index])
    if img is None:
        print("Error: Image not found or unable to load.")
        return
    original_image = img.copy()
    cv.imshow('Select Corners', img)
    # Set the mouse callback function
    cv.setMouseCallback('Select Corners', select_point)

In [69]:
# Mouse callback function
def select_point(event, x, y, flags, param):
    global points
    if event == cv.EVENT_LBUTTONDOWN:
        # Store the point coordinates
        points.append((x, y))
        # Draw a small circle at the selected point
        cv.circle(img, (x, y), 20, (0, 0, 255), -1)  # Red dot for points
        cv.imshow('Select Corners', img)

        # Automatically draw lines if four points have been selected
        if len(points) == 4:
            draw_square()
            warp_image()


In [70]:
def draw_square():
    # Draw lines between the four selected points
    for i in range(len(points)):
        # Draw a line between each consecutive point
        cv.line(img, points[i], points[(i + 1) % 4], (0, 255, 0), 2)  # Green lines
    cv.imshow('Select Corners', img)

In [71]:
def warp_image():
    # Convert points to numpy array
    src_pts = np.array(points, dtype='float32')

    # Calculate the width and height of the new image
    widthA = np.linalg.norm(src_pts[0] - src_pts[1])
    widthB = np.linalg.norm(src_pts[2] - src_pts[3])
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.linalg.norm(src_pts[0] - src_pts[3])
    heightB = np.linalg.norm(src_pts[1] - src_pts[2])
    maxHeight = max(int(heightA), int(heightB))

    # Define destination points for perspective transform
    dst_pts = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype='float32')

    # Compute the perspective transform matrix
    M = cv.getPerspectiveTransform(src_pts, dst_pts)

    # Apply the perspective transformation
    warped = cv.warpPerspective(original_image, M, (maxWidth, maxHeight))

    # Resize the image to 3500x3500 pixels
    warped = cv.resize(warped, (3500, 3500), interpolation=cv.INTER_LINEAR)


    # Rotate the image 90 degrees anticlockwise
    #warped = cv.rotate(warped, cv.ROTATE_90_COUNTERCLOCKWISE)

    # Show the warped image
    cv.imshow('Warped Image', warped)
    image = images[current_index]
    cv.imwrite(image.split('.')[0] + '-warped.jpg', warped)
    

In [72]:
images = glob.glob('Images/*.JPG')
current_index = 0
points = []

def handle_key(event):
    global current_index
    if event == 2:  # Left arrow key
        if current_index > 0:
            current_index -= 1
            load_image(current_index)
    elif event == 3:  # Right arrow key
        if current_index < len(images) - 1:
            current_index += 1
            load_image(current_index)
        else:
            print("End of images")

# Load the first image
load_image(current_index)

# Set the mouse callback function
cv.setMouseCallback('Select Corners', select_point)

# Main loop
while True:
    key = cv.waitKey(0) & 0xFF
    if key == 27:  # ESC key to exit
        break
    elif key in [2, 3]:  # Arrow keys
        points = []
        handle_key(key)

cv.destroyAllWindows()
cv.waitKey(1)

##PRESS RIGHT ARROW KEY TO GO TO NEXT IMAGE
##PRESS LEFT ARROW KEY TO GO TO PREVIOUS IMAGE
##PRESS ESC TO EXIT
##PRESS MOUSE LEFT CLICK TO SELECT 4 POINTS STARTING WITH TOP LEFT CORNER OF THE IMAGE AND GOING CLOCKWISE

-1